# Aiden's Project
----

Fall 2025

In [ ]:
# Importing the basics we need

import astropy.units as u
import astropy.constants as c
from astropy.coordinates import SkyCoord, search_around_sky
from astropy.time import Time
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import glob
#%matplotlib notebook
%matplotlib inline
from astropy.io import fits
from itertools import combinations
import pickle
from astropy.cosmology import WMAP9 as cosmo
import seaborn as sns
from astropy.table import Table


#Setting up matplotlib how I like it - you can change this to reflect your preferences.
plt.rcParams['figure.figsize'] = (10, 10)
plt.rc('axes', labelsize=27)
#plt.rc('axes', labelweight='bold')
plt.rc('axes', titlesize=27)
plt.rc('axes', titleweight='bold')
plt.rc('font', family='sans-serif')
plt.rcParams.update({'font.size': 20})

# Step 1: Reading the Data

When I send you files, they will always be easily read in as a pandas DataFrame. You'll learn as you go through physics and astronomy that every one has a different opinion on the best file format to exchange science data with. Everyone uses a different file type, method for reading files in, and data format. Everyone also belevies that thier method is the correct method and all other methods are inferior. I am no exception and refuse to change my file writing/reading methods.  

Pandas DataFrames are pretty standard in data science and learning to work with them will make your life a lot easier. Let's read in the files we need and run through an example.


In [ ]:
#File paths- CHANGE TO YOURS


#make sure the absolute path to your data preceds the file string.
all_ceers = pd.read_pickle('df_new.pkl')


Let's start with just printing out the Data Frame:

In [ ]:
all_ceers

Big Data! Let's take a look at the keyword in the Data Frame. 

In [ ]:
all_ceers.keys()

Lots of keys here. We will concentrate on RA (right ascension in decimal degrees), DEC (declination in decimal degrees), and PHOTOM_RED_SHIFT (the photometric red shift or line of sight distance) Let's build some arrays that carry just these around.

In [ ]:
ra_ceers = all_ceers['RA'].values
dec_ceers = all_ceers['DEC'].values
z_ceers = all_ceers['PHOTOM_RED_SHIFT'].values

You'll also be working with matplotlib to generate graphs. It has a lot of great functionality in Python. We can make a simple plot of CEERS using matplotlib's scatter plot:

In [ ]:
#Calling the scatter plot function
plt.scatter(ra_ceers,dec_ceers, s = .1)


#Adding x labels, y labels, and a title
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')
#By standard convention we always dsiplay RA backwards!
plt.gca().invert_xaxis()
plt.title('CEERS'); #End yout matplotlib statements with a semicolon to prevent it from printing every output

Another helpful tool for you will be boolean arrays. You can use thes to index larger arrays and slice them in to smaller ones. Let's build an array that looks at redshifts >5:

In [ ]:
bool1 = all_ceers['PHOTOM_RED_SHIFT'] >4

And index our values with the boolean:

In [ ]:
ra_g4 = ra_ceers[bool1]
dec_g4 = dec_ceers[bool1]

Let's plot them over our last plot, now in another color, so we can see where they are:

In [ ]:
#Calling the scatter plot function
plt.scatter(ra_ceers,dec_ceers, s = .1)

#Adding our new data and giving it a label for the legend
plt.scatter(ra_g4, dec_g4, c = 'r', s = .1, label = 'z>4')


#Adding x labels, y labels, and a title
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')
#Call the legend to display it
plt.legend()
#By standard convention we always dsiplay RA backwards!
plt.gca().invert_xaxis()
plt.title('CEERS'); #End yout matplotlib statements with a semicolon to prevent it from printing every output

Wow lots of sources! But let's refine our bins and look at sources that are also at redshifts <5. We will add another array:

In [ ]:
bool2 = all_ceers['PHOTOM_RED_SHIFT'] <5

And combine them with '&', which will grab the values that are true in both booleans. If we wanted all values that were true in both arrays instead, we would combine them with '|'. 

In [ ]:
ra_g4l5 = ra_ceers[bool1 & bool2] 
dec_g4l5 = dec_ceers[bool1 & bool2]

In [ ]:
#Calling the scatter plot function
plt.scatter(ra_ceers,dec_ceers, s = .1)

#Adding our new data and giving it a label for the legend
plt.scatter(ra_g4l5, dec_g4l5, c = 'r', s = .3, label = 'z>4')


#Adding x labels, y labels, and a title
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')
#Call the legend to display it
plt.legend()
#By standard convention we always dsiplay RA backwards!
plt.gca().invert_xaxis()
plt.title('CEERS'); #End yout matplotlib statements with a semicolon to prevent it from printing every output

Very cool! There's been a lot of suspicion about the super high redshift sources at z>10. Remake this plot for sources of redshifts 10-12. Are they in any particular area of CEERS? Do you notice any differences?

In [ ]:
#---YOUR WORK---

#Feel free to copy/paste some stuff from above and make your own plot!

# Step 2: Plotting CEERS with imshow()

While these plots are cool, they are limited in terms of the mathematical questions we can ask about the galaxy population. Instead, we turn to numpy's matricies to build something representative of the population. Here's some code where we make a grid and increase all the values in the grid from one to 0.

In [ ]:
#Creare some minimums
xmin = 0
xmax = 100
ymin = 0
ymax = 100
#Create x and y axes. If you change the grid resolution, you'll have to change these parameters
x_axis = np.linspace(xmin - 0.001, xmax + 0.001, num=100, endpoint=True)
y_axis = np.linspace(ymin - 0.001, ymax + 0.001, num=100, endpoint=True)



#Create an empty 100x100 matrix to store the data. You can change these values to change the map resolution,
#but this will increase your needed computing time!
data = np.matrix(np.zeros((100,100)))

#This loop structure lets us iterate over all points in the matrix
for xind, xval in enumerate(x_axis):
    for yind, yval in enumerate(y_axis):
        #Set the value for the data matrix at this point 
        data[xind, yind] += 1

In [ ]:
#We can just call imshow on the data to plot our matrix:
plt.imshow(data, origin = 'lower');

Boring! Let's instead increase the matrix values to sin(xy). We can use numpy to evaluate sin.

In [ ]:
#Create an empty 100x100 matrix to store the data. You can change these values to change the map resolution,
#but this will increase your needed computing time!
data = np.matrix(np.zeros((100,100)))

#This loop structure lets us iterate over all points in the matrix
for xind, xval in enumerate(x_axis):
    for yind, yval in enumerate(y_axis):
        #Set the value for the data matrix at this point 
        data[xind, yind] += np.sin(xind * yind)

In [ ]:
#We can just call imshow on the data to plot our matrix:
plt.imshow(data, origin = 'lower');

Much more fun. Try increasing the value at every

You'll need to turn your ra/dec arryas in to these matricies. Here's some code to re-make the plot, now with the x_axis and y_axis reflective of the values in CEERS.

In [ ]:
#Creare some minimums and maximums to reflect the CEERS filed. You may need to play around with adding/subtracting
#values to make the data matrix reflect CEERS


xmin = np.min(ra_ceers)
xmax = np.max(ra_ceers)
ymin = np.min(dec_ceers)
ymax = np.max(dec_ceers)
#Create x and y axes. If you change the grid resolution, you'll have to change these parameters
x_axis = np.linspace(xmin - 0.001, xmax + 0.001, num=100, endpoint=True)
y_axis = np.linspace(ymin - 0.001, ymax + 0.001, num=100, endpoint=True)



#Create an empty 100x100 matrix to store the data. You can change these values to change the map resolution,
#but this will increase your needed computing time!
data = np.matrix(np.zeros((100,100)))

#This loop structure lets us iterate over all points in the matrix
for xind, xval in enumerate(x_axis):
    for yind, yval in enumerate(y_axis):
        #Set the value for the data matrix at this point 
        data[xind, yind] += xval * yval

#We can just call imshow on the data to plot our matrix:
plt.imshow(data, origin = 'lower');


Next, edit the loop that changes the value of the data matrix to increase the value in the array by 1 only if a CEERS source falls within that pixel.

In [ ]:
#Creare some minimums and maximums to reflect the CEERS filed. You may need to play around with adding/subtracting
#values to make the data matrix reflect CEERS


xmin = np.min(ra_ceers)
xmax = np.max(ra_ceers)
ymin = np.min(dec_ceers)
ymax = np.max(dec_ceers)
#Create x and y axes. If you change the grid resolution, you'll have to change these parameters
x_axis = np.linspace(xmin - 0.001, xmax + 0.001, num=100, endpoint=True)
y_axis = np.linspace(ymin - 0.001, ymax + 0.001, num=100, endpoint=True)



#Create an empty 100x100 matrix to store the data. You can change these values to change the map resolution,
#but this will increase your needed computing time!
data = np.matrix(np.zeros((100,100)))


#This loop structure lets us iterate over all points in the matrix
for xind, xval in enumerate(x_axis):
    for yind, yval in enumerate(y_axis):
        #---------------------
        #Edit the code down here and make your plot
        if.....
            data[xind, yind] += ....
        #---------------------


In [ ]:
#Now you get to customize this! add a color bar with a label, pick a color map, change the
#image size, and make the x and y axis labels, and make sure to flip the RA axis (like we did with the scatter plot)


plt.imshow(data, origin = 'lower');


# Step 3: Adding the LRDs

This is the paper we will use for the CEERS LRDs

https://ui.adsabs.harvard.edu/abs/2025ApJ...986..126K/abstract

The data from this paper is hosted at: https://github.com/dalekocevski/Kocevski24

You'll take a few steps here:

- read in the data file for LRDs. Make arrays for their RA, DEC, and redshift
  
- create a scatter plot of the LRDs

- plot the LRDs over your data matrix for CEERS



# Step 4: Redshift Bins

Lastly, we will break the CEERS and LRD data sets in to redshift bins. You'll want bins that span about 1 redshift. This will take a few steps:

- Make histograms from matplotlib of the redhsift distribution of CEERS and the LRDs. This will give us a better idea of what the correct redshifts bins to use are.

- Modify your CEERS code to only add galaxies to the data matrix if they fall within the target redhsift bin

- Over plot the LRDs that fall in this redshift bin

# Step 5: Do LRDs fall in over-dense or under-dense environments?

Now we get to answer our question! Do LRDs live in pixels with more or less sources?

- find the mean value of your CEERS matrix. Plot a histogram of your pixel values. 

- calculate the typical pixel value of the pixels containing LRDs. What is typical value for LRDs?

- make the same histograms for your LRDs

- Do LRDs tend to be in pixels that are more or less dense than the background population?

Next Steps:

If we finish early or want to continue the project next semester, these are some of the things we could do with your data:

- weighted gaussian smoothing

- comparison AGN https://ui.adsabs.harvard.edu/abs/2025ApJ...986..165T/abstract

- NNN calculations